In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

url='https://raw.githubusercontent.com/justmarkham/DAT8/master/data/u.data'
# load the movielens dataset
columns = ['user_id','item_id','rating','timestamp']
df=pd.read_csv(url,sep='\t',names=columns)
df.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [4]:
#create a user-item matrix

user_item_matrix=df.pivot_table(index='user_id',columns='item_id',values='rating')
user_item_matrix.fillna(0,inplace=True)
user_item_matrix.head()


item_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
#calculate the similarity between users
user_similarity=cosine_similarity(user_item_matrix)
user_similarity_df=pd.DataFrame(user_similarity,index=user_item_matrix.index,columns=user_item_matrix.index)
user_similarity_df.head()

user_id,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
user_id,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.166931,0.047460,0.064358,0.378475,0.430239,0.440367,0.319072,0.078138,0.376544,...,0.369527,0.119482,0.274876,0.189705,0.197326,0.118095,0.314072,0.148617,0.179508,0.398175
2,0.166931,1.000000,0.110591,0.178121,0.072979,0.245843,0.107328,0.103344,0.161048,0.159862,...,0.156986,0.307942,0.358789,0.424046,0.319889,0.228583,0.226790,0.161485,0.172268,0.105798
3,0.047460,0.110591,1.000000,0.344151,0.021245,0.072415,0.066137,0.083060,0.061040,0.065151,...,0.031875,0.042753,0.163829,0.069038,0.124245,0.026271,0.161890,0.101243,0.133416,0.026556
4,0.064358,0.178121,0.344151,1.000000,0.031804,0.068044,0.091230,0.188060,0.101284,0.060859,...,0.052107,0.036784,0.133115,0.193471,0.146058,0.030138,0.196858,0.152041,0.170086,0.058752
5,0.378475,0.072979,0.021245,0.031804,1.000000,0.237286,0.373600,0.248930,0.056847,0.201427,...,0.338794,0.080580,0.094924,0.079779,0.148607,0.071459,0.239955,0.139595,0.152497,0.313941


In [12]:
def predict_ratings(user_id, user_item_matrix,user_similarity_df):
    similar_users=user_similarity_df[user_id]
    weighted_ratings=np.dot(similar_users,user_item_matrix)
    sum_of_weights=np.abs(similar_users).sum()
    predicted_ratings=weighted_ratings/sum_of_weights
    return pd.Series(predicted_ratings,index=user_item_matrix.columns)

user_id=10
predict_ratings=predict_ratings(user_id,user_item_matrix,user_similarity_df)
predict_ratings.sort_values(ascending=False).head(10)

item_id
50     3.348947
100    2.873344
174    2.810063
98     2.650287
181    2.613021
127    2.530208
56     2.516140
172    2.371805
1      2.217906
204    2.126573
dtype: float64

content based recomendation system


In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

movies={
    'title': ['Movie A','Movie B','Movie C','Movie D'],
    'genres': ['Action|Adventure','Adventure|Comedy','Comedy|Romance','Action|Thriller']

}
movie_df=pd.DataFrame(movies)
movie_df

,title,genres
0,Movie A,Action|Adventure
1,Movie B,Adventure|Comedy
2,Movie C,Comedy|Romance
3,Movie D,Action|Thriller


In [16]:
#vectorize
tfidf =TfidfVectorizer(token_pattern='[a-zA-Z0-9_]+')
tfidf_matrix=tfidf.fit_transform(movie_df['genres'])
tfidf_feature_names=tfidf.get_feature_names_out()
pd.DataFrame(tfidf_matrix.toarray(),columns=tfidf_feature_names,index=movie_df['title'])

,action,adventure,comedy,romance,thriller
title,,,,,
Movie A,0.707107,0.707107,0.000000,0.000000,0.000000
Movie B,0.000000,0.707107,0.707107,0.000000,0.000000
Movie C,0.000000,0.000000,0.619130,0.785288,0.000000
Movie D,0.619130,0.000000,0.000000,0.000000,0.785288


In [18]:
#calculate the similarity between
cosine_sim=cosine_similarity(tfidf_matrix)
cosine_sim_df=pd.DataFrame(cosine_sim,index=movie_df['title'])
cosine_sim_df

,0,1,2,3
title,,,,
Movie A,1.000000,0.500000,0.000000,0.437791
Movie B,0.500000,1.000000,0.437791,0.000000
Movie C,0.000000,0.437791,1.000000,0.000000
Movie D,0.437791,0.000000,0.000000,1.000000


In [26]:
#recommend movies similar to given movie
def recommend_movies(title,cosine_sim_df,num_recommendations=3):
    similar_score=cosine_sim_df[title]
    similar_movies=similar_score.sort_value(ascending=False).iloc[1:num_recommendations+1]
    return similar_movies

recommendations=recommend_movies('Movie A',cosine_sim_df)
recommendations

KeyError: 'Movie A'